In [ ]:
from slippi_db import upload_lib

In [ ]:
prod_slp = upload_lib.get_db('prod', 'slp')
prod_slp = list(prod_slp.find())

In [ ]:
prod_parsed_db = upload_lib.get_db('prod', upload_lib.PQ)
prod_pq = list(prod_parsed_db.find())

In [ ]:
biggest = sorted(prod_slp, key=lambda doc: doc['original_size'])[-10:]

In [ ]:
prod_meta = list(upload_lib.get_db('prod', 'meta').find())

In [ ]:
key_to_meta = {meta['key']: meta for meta in prod_meta}

In [ ]:
for doc in biggest:
    print(key_to_meta[doc['key']])

In [ ]:
import pandas as pd

In [ ]:
parsed_keys = set(doc['key'] for doc in prod_pq if not doc['failed'])

In [ ]:
bogus = [doc['key'] for doc in prod_pq if doc['failed']]

In [ ]:
len(bogus)

In [ ]:
key_to_parse = {doc['key']: doc for doc in prod_pq}

In [ ]:
key_to_parse[bogus[0]]

In [ ]:
reasons = set(key_to_parse[key]['reason'] for key in bogus)

In [ ]:
import collections
reason_to_key = collections.defaultdict(list)
for key in bogus:
    reason_to_key[key_to_parse[key]['reason']].append(key)

In [ ]:
for reason, keys in reason_to_key.items():
    print(f'{reason}: {len(keys)}')

In [ ]:
event_type_61_keys = reason_to_key["<class 'ValueError'>: 61 is not a valid EventType"]

In [ ]:
prod_raw = list(upload_lib.get_db('prod', upload_lib.RAW).find())

In [ ]:
key_to_raw = {raw['key']: raw for raw in prod_raw}
key_to_slp = {slp['key']: slp for slp in prod_slp}

In [ ]:
key = event_type_61_keys[0]

In [ ]:
raw_files = set(key_to_raw[key_to_slp[key]['raw_key']]['filename'] for key in event_type_61_keys)

In [ ]:
raw_files

In [ ]:
parsed_without_meta = []

for key, parse in key_to_parse.items():
    if parse['failed']:
        continue
    
    meta = key_to_meta.get(key)
    if not meta:
        parsed_without_meta.append(key)
    
    invalid = meta.get('invalid') or meta.get('failed')
    if invalid:
        parsed_without_meta.append(key)

len(parsed_without_meta)

In [ ]:
deletions = [pymongo.DeleteOne({'key': key}) for key in parsed_without_meta]
prod_parsed_db.bulk_write(deletions)

In [ ]:
valid_meta_keys = [
    key for key, meta in key_to_meta.items()
    if not (meta.get('invalid') or meta.get('failed'))
]

In [ ]:
import pymongo

In [ ]:

pymongo.DeleteMany